`Model Evaluation`

During this notebook we're following the Workflow from the updation of `config.yaml` to `app.py` 

In [1]:
import os
%cd "C:\Users\marou\OneDrive\Desktop\soloLeveling\Projects\Text-Summarization"

C:\Users\marou\OneDrive\Desktop\soloLeveling\Projects\Text-Summarization
